In [ ]:
from numpy import genfromtxt

from jax import random
import jax.numpy as jnp

import numpyro
import numpyro.distributions as dist
from numpyro import infer
numpyro.set_host_device_count(2) # let's use 2 cores!

import matplotlib.pyplot as plt
import arviz as az

# Linear Regression w/ Outliers

This notebook ended up mirroring much of the work in [Dan Foreman-Mackey's blog post](https://dfm.io/posts/intro-to-numpyro/).

Let's again use some synthetic data $(x_i,y_i)$, where $\{x_i\}$ are known with negligible uncertainty, and $\{y_i\}$ values have variable uncertainties $\{\sigma_{yi}\}$.

To deal with outliers we need to define a *generative model* for both the good data (foreground) and the bad (background).  Each datum can either be from the foreground or the background, and since we don't know *a priori*, we need to introduce discrete parameters $q_i$, where $q_i=0$ indicates datum $i$ is background, and $q_i=1$ if foreground.  We'll need to infer the probablility $P_b$ that any particular datum is part of the background.  Since we need a generative model for the background, let's assume it's normally distributed with mean and standard deviation $\mu_b$ and $\sigma_b$, independent of $x$.

We have now added $N+3$ extra parameters $(\{q_i\}_{i=1}^N, P_b, \mu_b, \sigma_b)$ to deal with outliers, and the likelihood takes the form

$$
\mathcal{L} = p\left(\{y_i\}_{i=1}^N|m,b,\{q_i\}_{i=1}^N,\mu_b,\sigma_b,I\right)
$$

or, broken down into foreground and background

$$
\mathcal{L} = \prod_i \left[p_\mathrm{fg}(\{y_i\}|m,b,I)\right]^{q_i} \left[p_\mathrm{bg}(\{y_i\}|\mu_b, \sigma_b, I)\right]^{1-q_i},
$$

where

$$
p_\mathrm{fg} = \frac{1}{\sqrt{2\pi\sigma_{yi}^2}} \exp \left(-\frac{(y_i - mx_i - b)^2}{2\sigma_{yi}^2}\right) \\
p_\mathrm{bg} = \frac{1}{\sqrt{2\pi(\sigma_b^2 + \sigma_{yi}^2})} \exp \left(-\frac{(y_i - \mu_b)^2}{2(\sigma_b^2 +\sigma_{yi}^2)}\right)
$$

Since we are specifying a probability $P_b$ of each data point being an outlier, we need to incorporate this binomial probability in our prior

$$
p(m,b,\{q_i\},P_b,\mu_b,\sigma_b|I) = p(\{q_i\}|P_b,I)p(m,b,P_b,\mu_b,\sigma_b|I)
$$

where $p(\{q_i\}|P_b,I)$ is the binomial probability

$$
p(\{q_i\}|P_b,I) = \prod_i \left(1 - P_b\right)^{q_i}P_b^{1-q_i}
$$

We could sample in the (very) high-dimensional space, sampling both the continuous variables and the discrete ones.  However, we can also _marginalize_ over the $\{q_i\}$'s analytically, and save our MCMC some work.

Focusing on our likelihood

$$
p\left(\{y_i\}_{i=1}^N|m,b,\{q_i\}_{i=1}^N,\mu_b,\sigma_b\right) = \prod_i p(y_i|m,b,q_i,\mu_b,\sigma_b),
$$

we can integrate over (actually sum, since it's discrete) the unknown $q_i$'s to account for their uncertainty.  To do so we need to specify our prior $p(q_i)$ and we are left with:

$$
p\left(\{y_i\}_{i=1}^N|m,b,\{q_i\}_{i=1}^N,\mu_b,\sigma_b\right) = \prod_i \left [p(q_i=0) p(y_i|m,b,\mu_b,\sigma_b, q_i=0) + p(q_i=1) p(y_i|m,b,\mu_b,\sigma_b, q_i=1)\right],
$$

which is a _mixture model_!

## Foreground-only Model

First let's repeat our previous analysis that assumed all data to be from the foreground model, i.e., following the linear relationship.  Before we let numpyro's Normal distribution implicitly vectorize the likelihood, which assumes each data point to be independent.  In general, the way to tell numpyro that values along a particular dimensions should be treated independently is to use a _plate_.  We'll use that here.

In [ ]:
def linear_model(x=None, y=None, σ_y=None):
...

Let's look at the lines corresponding to a few posterior draws to get a sense of the uncertainty in the line we're fitting.

Before improving our model, let's exercise one of numpyro superpowers, which is _predictive_ sampling, either by sampling our prior, or _posterior predictive sampling_, where we "predict" the observations based on our posterior estimates of the model parameters. 

This makes it painfully obvious that our model is _not_ a good descriptor of the data.

## Foreground and Background Model

In [ ]:
import jax

In [ ]:
def linear_model_w_outliers(x=None, y=None, σ_y=None):
...

Let's take a look at the distribution probabilities of each point being in the foreground.

In [ ]:
...